In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-12-13 03:42:01--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  20.5MB/s    in 1.2s    

2024-12-13 03:42:03 (20.5 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [11]:
# Remove users with less than 200 ratings
user_rating_counts = df_ratings['user'].value_counts()
valid_users = user_rating_counts[user_rating_counts >= 200].index
df_ratings = df_ratings[df_ratings['user'].isin(valid_users)]

# Check if there are any valid ratings left after filtering
print(f"Users left after filtering: {df_ratings['user'].nunique()}")

# Remove books with less than 100 ratings
book_rating_counts = df_ratings['isbn'].value_counts()
valid_books = book_rating_counts[book_rating_counts >= 100].index
df_ratings = df_ratings[df_ratings['isbn'].isin(valid_books)]

# Check if there are any valid books left after filtering
print(f"Books left after filtering: {df_ratings['isbn'].nunique()}")

# Drop any existing 'title' column to avoid merge conflicts
df_ratings.drop(columns=['title'], errors='ignore', inplace=True)

# Now merge ratings with book details to get book titles (merge only once)
df_ratings = pd.merge(df_ratings, df_books[['isbn', 'title']], on='isbn', how='left')

# Check if the merge worked
print(f"Columns after merge: {df_ratings.columns}")
print(f"Number of records after merge: {len(df_ratings)}")

# Drop any redundant 'title' columns if they exist
df_ratings.drop(columns=[col for col in df_ratings.columns if col != 'user' and col != 'isbn' and col != 'rating' and col != 'title'], inplace=True)

# Check if the cleanup was successful
print(f"Columns after cleanup: {df_ratings.columns}")
print(f"Number of records after cleanup: {len(df_ratings)}")

# Now proceed to create the user-item rating matrix
rating_matrix = df_ratings.pivot(index='user', columns='title', values='rating').fillna(0)

# Check if the matrix was created correctly
print(f"Rating matrix shape: {rating_matrix.shape}")
print(rating_matrix.head())

# If rating_matrix is not empty, proceed with KNN
if rating_matrix.shape[0] > 0 and rating_matrix.shape[1] > 0:
    # Convert to sparse matrix (optional for performance)
    from scipy.sparse import csr_matrix
    rating_matrix_sparse = csr_matrix(rating_matrix.values)

    # Initialize the KNN model
    from sklearn.neighbors import NearestNeighbors
    knn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')
    knn.fit(rating_matrix_sparse.T)  # Fit on the transposed matrix (books as rows)
else:
    print("The rating matrix is empty. Please check your filtering steps and data.")


Users left after filtering: 0
Books left after filtering: 0
Columns after merge: Index(['user', 'isbn', 'rating', 'title'], dtype='object')
Number of records after merge: 0
Columns after cleanup: Index(['user', 'isbn', 'rating', 'title'], dtype='object')
Number of records after cleanup: 0
Rating matrix shape: (0, 0)
Empty DataFrame
Columns: []
Index: []
The rating matrix is empty. Please check your filtering steps and data.


In [12]:
# Merge ratings with book details to get book titles (correct column handling)
df_ratings = pd.merge(df_ratings, df_books[['isbn', 'title']], on='isbn', how='left')

# Check the column names to ensure you're using the correct 'title' column
print(df_ratings.columns)

# If necessary, rename the 'title_y' column to just 'title'
df_ratings.rename(columns={'title_y': 'title'}, inplace=True)

# Create a user-item rating matrix
rating_matrix = df_ratings.pivot(index='user', columns='title', values='rating').fillna(0)

# Now proceed with the rest of your code...


Index(['user', 'isbn', 'rating', 'title_x', 'title_y'], dtype='object')


In [21]:
def get_recommends(book=""):
    # Check if the book exists in the rating matrix
    if book not in rating_matrix.columns:
        return [book, []]  # Return the book title and an empty list if not found

    # Find the index of the book in the matrix
    book_index = rating_matrix.columns.get_loc(book)

    # Fit the KNN model on the rating matrix (to avoid repeated fitting)
    knn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')
    knn.fit(rating_matrix_sparse.T)

    # Get distances and indices of the nearest neighbors
    distances, indices = knn.kneighbors(rating_matrix_sparse.T[book_index], n_neighbors=6)

    # Get the recommended book titles and distances
    recommended_books = []
    for i in range(1, len(distances[0])):  # Skip the first book (itself)
        book_title = rating_matrix.columns[indices[0][i]]
        recommended_books.append([book_title, distances[0][i]])

    return [book, recommended_books]


In [22]:
def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

    if not recommends[1]:  # If no recommendations are found
        print("Book not found in the dataset.")
        return

    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False

    # Define the recommended books and distances to compare
    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]

    for i in range(4):  # Check for 4 books as per the test case
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False

    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")
